In [19]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans 
from sklearn.cluster import DBSCAN
from sklearn import metrics
from scipy.spatial.distance import cdist
from math import comb
from itertools import combinations

In [3]:
#initial data view
data = pd.read_excel('sampled_5per_data.xlsx')
y_num = np.array(data['Length_Of_Stay'])
y_disc = np.array(data['LOS_Discrete'])
data.head()

,Unnamed: 0,Age_Group,Gender,Race,Type_Of_Admission,Diagnosis_Description,Procedure_Description,APR_DRG_Description,APR_MDC_Description,APR_Severity_Of_Illness_Description,APR_Risk_Of_Mortality,APR_Medical_Surgical_Description,Payment_Typology_1,Is_Emergency_Department_Indicator,Length_Of_Stay,LOS_Discrete
0,0,70 or Older,F,White,Emergency,"STRESS FRACTURE, INITIAL ENCOUNTER",BONE FIXATION (EXCLUDING EXTREMITIES),CERVICAL SPINAL FUSION AND OTHER BACK OR NECK ...,DISEASES AND DISORDERS OF THE MUSCULOSKELETAL ...,Minor,Minor,Surgical,Blue Cross/Blue Shield,True,1,1
1,1,50 to 69,F,Other Race,Elective,OTHER SPECIFIED AND UNSPECIFIED CIRCULATORY DI...,LIGATION AND EMBOLIZATION OF VESSELS,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,DISEASES AND DISORDERS OF THE CIRCULATORY SYSTEM,Moderate,Minor,Surgical,Medicaid,False,1,1
2,2,50 to 69,F,White,Elective,OBESITY,DIAPHRAGMATIC HERNIA REPAIR,PROCEDURES FOR OBESITY,"ENDOCRINE, NUTRITIONAL AND METABOLIC DISEASES ...",Minor,Minor,Surgical,Blue Cross/Blue Shield,False,1,1
3,3,70 or Older,F,White,Elective,OSTEOARTHRITIS,ARTHROPLASTY OF OTHER JOINT (EXCLUDING KNEE AN...,SHOULDER AND ELBOW JOINT REPLACEMENT,DISEASES AND DISORDERS OF THE MUSCULOSKELETAL ...,Minor,Minor,Surgical,Medicare,False,1,1
4,4,50 to 69,F,White,Emergency,HYPERTENSION WITH COMPLICATIONS AND SECONDARY ...,ISOLATION PROCEDURES,HYPERTENSION,DISEASES AND DISORDERS OF THE CIRCULATORY SYSTEM,Minor,Minor,Medical,Medicare,True,1,1


In [4]:
#discard features that won't be used
data = data.drop(data.columns[0], axis=1)
data = data.drop(columns=['Diagnosis_Description','Procedure_Description','APR_DRG_Description'])
data = data.drop(columns=['Length_Of_Stay','LOS_Discrete'])
features = list(data.columns)
features

['Age_Group',
 'Gender',
 'Race',
 'Type_Of_Admission',
 'APR_MDC_Description',
 'APR_Severity_Of_Illness_Description',
 'APR_Risk_Of_Mortality',
 'APR_Medical_Surgical_Description',
 'Payment_Typology_1',
 'Is_Emergency_Department_Indicator']

In [5]:
# Generate combinations of features that will be used
comb_features = ['APR_MDC_Description', 'APR_Severity_Of_Illness_Description',
                 'APR_Risk_Of_Mortality', 'APR_Medical_Surgical_Description']

fix_features = ['Age_Group','Gender','Race','Type_Of_Admission',
                'Payment_Typology_1', 'Is_Emergency_Department_Indicator']

initial_dict = {}
id = 0
for r in range(1, len(comb_features) + 1):
    cur_list = list(combinations(comb_features, r))
    for l in cur_list:
        initial_dict[id] = list(l)
        id += 1
        
combinations_dict = {}
for key, value in initial_dict.items():
    combinations_dict[key] = fix_features + value
    
combinations_dict


{0: ['Age_Group',
  'Gender',
  'Race',
  'Type_Of_Admission',
  'Payment_Typology_1',
  'Is_Emergency_Department_Indicator',
  'APR_MDC_Description'],
 1: ['Age_Group',
  'Gender',
  'Race',
  'Type_Of_Admission',
  'Payment_Typology_1',
  'Is_Emergency_Department_Indicator',
  'APR_Severity_Of_Illness_Description'],
 2: ['Age_Group',
  'Gender',
  'Race',
  'Type_Of_Admission',
  'Payment_Typology_1',
  'Is_Emergency_Department_Indicator',
  'APR_Risk_Of_Mortality'],
 3: ['Age_Group',
  'Gender',
  'Race',
  'Type_Of_Admission',
  'Payment_Typology_1',
  'Is_Emergency_Department_Indicator',
  'APR_Medical_Surgical_Description'],
 4: ['Age_Group',
  'Gender',
  'Race',
  'Type_Of_Admission',
  'Payment_Typology_1',
  'Is_Emergency_Department_Indicator',
  'APR_MDC_Description',
  'APR_Severity_Of_Illness_Description'],
 5: ['Age_Group',
  'Gender',
  'Race',
  'Type_Of_Admission',
  'Payment_Typology_1',
  'Is_Emergency_Department_Indicator',
  'APR_MDC_Description',
  'APR_Risk_Of_Mo

In [6]:
#describe possible tuning configurations

num_clusters_list = list(range(3, 11))
km_methods = ['builtin','developed']
init_methods = ['random','k-means++']
num_iter = 300

In [7]:
#define pairwise distance function
def pairwise_dist(x, y):
    x_sq = np.sum(x**2, axis=1, keepdims=True)
    y_sq = np.sum(y**2, axis=1, keepdims=True)
    xy = np.dot(x, y.T)
    distances = np.sqrt(x_sq + y_sq.T - 2 * xy)
    return distances

In [8]:
#beta-cv score calculator
def beta_cv(X,labels):

    unique_clusters = np.unique(labels)
    num_clusters = len(unique_clusters)
    inter_dist = 0
    inter_n = 0
    for i in range(num_clusters-1):
        for j in range(i+1, num_clusters):

            i_indices = np.where(labels == unique_clusters[i])[0]
            j_indices = np.where(labels == unique_clusters[j])[0]

            distances = cdist(X[i_indices], X[j_indices])
            inter_dist += np.sum(distances)

            inter_n += (len(i_indices)*len(j_indices))

    intra_dist = 0
    intra_n = 0
    for i in range(num_clusters):

        i_indices = np.where(labels == unique_clusters[i])[0]

        distances = cdist(X[i_indices], X[i_indices])
        intra_dist += (np.sum(distances)/2)

        intra_n += comb(len(i_indices), 2)

    beta_cv = (intra_dist/intra_n)/(inter_dist/inter_n)
    return beta_cv


In [13]:
#function to use sklearn built-in kmeans algorithm
def builtin_method(X, num_clusters, init_method):   
    
    kmeans = KMeans(n_clusters=num_clusters,
                    init=init_method,
                    random_state=13
                    max_iter = num_iter)
    kmeans.fit(X)

    labels = kmeans.labels_
    
    return labels

In [16]:
#function to use developed kmeans algorithm
def developed_method(X, num_clusters, init_method,num_iter):
    
    np.random.seed(13)
    
    if init_method == 'random':
        indicies = np.random.choice(X.shape[0], num_clusters, replace=False)
        centers=X[indicies]

    else:   
        num_sample = int(0.01 * X.shape[0])
        sampled_points = X[np.random.choice(X.shape[0], num_sample, replace=False)]

        first_center_index = np.random.choice(sampled_points.shape[0])
        centers = [sampled_points[first_center_index]]

        for k in range(2, num_clusters+1):

            min_distances = np.zeros(sampled_points.shape[0])

            for i, point in enumerate(sampled_points):
                distances = [np.sum((point - center)**2) for center in centers]
                min_distances[i] = min(distances)

            max_distance_index = np.argmax(min_distances)

            centers.append(sampled_points[max_distance_index])

        centers=np.array(centers)
         
    assignments = np.zeros(X.shape[0])

    it = 0
    prev_loss = float('inf')
    while it < num_iter:
        distances = pairwise_dist(X, centers)
        assignments = np.argmin(distances, axis=1)

        unique_elements = np.unique(assignments,axis=0)
        new_centers = np.array([np.average(X[assignments == a],axis=0) for a in unique_elements])
        centers = new_centers

        if len(centers) < num_clusters:
            new_cluster_num = num_clusters - len(centers)

            add_center = X[np.random.choice(X.shape[0], new_cluster_num, replace=False)]

            centers = np.concatenate((centers,add_center),axis=0)

        distances = pairwise_dist(X, centers)
        assignments = np.argmin(distances, axis=1)

        unique_elements = np.unique(assignments,axis=0)
        new_centers = np.array([np.average(X[assignments == a],axis=0) for a in unique_elements])
        centers = new_centers

        dist = pairwise_dist(X, centers)
        sq_dist = dist**2
        extract = sq_dist[np.arange(X.shape[0]),assignments]
        new_loss = np.sum(extract)

        if prev_loss-new_loss == 0:
            break

        prev_loss = new_loss
        it += 1

    labels = assignments
    
    return labels

In [212]:
#find results for each tuning configuration and save them to an excel file 

max_sil = 0
max_sil_params = {}
min_beta = 1
min_beta_params = {}
main_list = []

for feats_id in combinations_dict.keys():
    for num_clusters in num_clusters_list:
        for method in km_methods:
            for init_method in init_methods:
                
                data_used = data[combinations_dict[feats_id]]
                data_encode = pd.get_dummies(data,drop_first=True)
                data_encode = data_encode.astype(int)
                X = data_encode.values

                if method == 'builtin':
                    labels = builtin_method(X, num_clusters, init_method)
                
                else:
                    labels = developed_method(X, num_clusters, init_method)
     
                sil_score = metrics.silhouette_score(X, labels)
                beta_score = beta_cv(X, labels)
            
                if sil_score > max_sil:
                    max_sil = sil_score
                    max_sil_params = {'features': combinations_dict[feats_id],
                                      'num_clusters':num_clusters,
                                      'method':method, 
                                      'init method':init_method}
                if beta_score < min_beta:
                    min_beta = beta_score
                    min_beta_params = {'features': combinations_dict[feats_id],
                                      'num_clusters':num_clusters,
                                      'method':method, 
                                      'init method':init_method}
                    
                feats = ','.join(combinations_dict[feats_id])
                main_list.append([feats_id, num_clusters, method, init_method, sil_score, beta_score, feats])
                print('one search done')

df = pd.DataFrame(main_list, columns=['features id', 'num clusters', 'mehod', 'init method', 'sil score (max)', 'beta score (min)', 'features'])
df.to_excel('search_results.xlsx',index=False)

print('Maximum silhouette score: ', max_sil)
print('Its params: ', max_sil_params)
print()
print('Minimum beta-cv score: ', min_beta)
print('Its params: ', min_beta_params)

C:\Users\cozmemis3\AppData\Local\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


one search done


C:\Users\cozmemis3\AppData\Local\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


one search done
one search done
one search done


C:\Users\cozmemis3\AppData\Local\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


...

one search done
one search done
one search done


C:\Users\cozmemis3\AppData\Local\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


one search done


C:\Users\cozmemis3\AppData\Local\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


one search done
one search done
one search done
Maximum silhouette score:  0.12943932671133784
Its params:  {'features': ['Age_Group', 'Gender', 'Race', 'Type_Of_Admission', 'Payment_Typology_1', 'Is_Emergency_Department_Indicator', 'APR_MDC_Description'], 'num_clusters': 7, 'method': 'builtin', 'init method': 'k-means++'}

Minimum beta-cv score:  0.7557834743801913
Its params:  {'features': ['Age_Group', 'Gender', 'Race', 'Type_Of_Admission', 'Payment_Typology_1', 'Is_Emergency_Department_Indicator', 'APR_MDC_Description'], 'num_clusters': 10, 'method': 'developed', 'init method': 'k-means++'}


In [200]:
df = pd.DataFrame(main_list, columns=['features id', 'num clusters', 'mehod', 'init method', 'sil score (max)', 'beta score (min)', 'features'])
df.to_excel('search_results.xlsx',index=False)

print('Maximum silhouette score: ', max_sil)
print('Its params: ', max_sil_params)
print()
print('Minimum beta-cv score: ', min_beta)
print('Its params: ', min_beta_params)

Maximum silhouette score:  0.12115558465917793
Its params:  {'features': ['Age_Group', 'Gender', 'Race', 'Type_Of_Admission', 'APR_MDC_Description'], 'num_clusters': 3, 'method': 'developed', 'init method': 'random'}

Minimum beta-cv score:  0.825669194494316
Its params:  {'features': ['Age_Group', 'Gender', 'Race', 'Type_Of_Admission', 'APR_MDC_Description'], 'num_clusters': 4, 'method': 'builtin', 'init method': 'k-means++'}
